In [26]:
from datasets import load_dataset, concatenate_datasets

def format_for_belar(row):
    row["context"] = row["selftext"]
    row["question"] = row["title"]
    row['answers'] = row["answers"]["text"]
    return row
    
d = load_dataset("eli5")
ds = d['test_eli5'].map(format_for_belar, batched=False)
ds = d_tmp.select_columns(["context", "question", "answers"])

Found cached dataset eli5 (/home/jjmachan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


  0%|          | 0/9 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jjmachan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-b6864f61633d1e41.arrow


In [34]:
ds.shape

(100, 3)

In [35]:
ds = ds.shuffle().select(range(100))
ds.shape

(100, 3)

In [44]:
from langchain.llms import OpenAI

llm = OpenAI()
prompt = """
{context}
with the context above xplain like I'm five: {question}
"""
i = 16
print(
    llm(prompt.format(
        context=ds[i]['context'],
        question=ds[i]['question'])
       )
)


Acronyms are words made out of the first letters of other words. For example, the word "radar" is made out of the first letters of "radio detection and ranging." The people who invented these words made them so that they could be said quickly, but it is also lucky that we can say them as a noun, or a word for a person, place, or thing.


In [45]:
ds[i]["question"], ds[i]["answers"]

('When acronyms are used as words such as radar, laser, sonar, scuba, are they derived with the intent of being used as words or is it just happy luck that we can say them as a noun?',
 ["A lot of them are specifically named so they can be said as a word. I'm sure there are some that weren't, too.\n\nDo you think the **U**niting and **S**trengthening **A**merica by **P**roviding **A**ppropriate **T**ools **R**equired to **I**ntercept and **O**bstruct **T**errorism Act (the USA PATRIOT Act) was named that way by accident?",
  'If a the abbreviations of a group of words can be pronounced, they are an acronym.  Examples; SCUBA, NASA, SONAR, etc.  If they can not be pronounced they are an initialism. Examples; FBI, CIA, DVD.\n\n_URL_0_'])

In [ ]:
def get_answers(row):
    row["answer_generated"] = llm(prompt.format(
        context=row['context'],
        question=row['question'])
       )
    return row
    
ds.map(get_answers, batched=False)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]